# Código do dashboard
## Autores: Agnaldo B. Júnior e Pedro H. C. dos Santos

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import psycopg2

/home/pedro/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


### Questão 5a:

In [2]:
def getHelpfulReviews(asin, con):
    cursor = con.cursor()
    cursor.execute('''select * from (
                  (select * from products join reviews r on products.asin = r.product_asin where asin='{}' order by helpful DESC , rating DESC LIMIT 5)
                  union all
                  (select * from products join reviews r on products.asin = r.product_asin where asin='{}' order by rating DESC ,helpful DESC  LIMIT 5)
              ) result;'''.format(asin,asin))
    rows = cursor.fetchall()
    cursor.close()
    con.close()
    return rows

### Questao 5b:

In [3]:
def getSimilarsWithMostSalesrank(asin, con):
    cursor = con.cursor()
    cursor.execute('''select * from (similars JOIN products ON similars.asin_1 = products.asin) AS result
    JOIN products ON result.asin_2 = products.asin  where asin_1='{}' and result.salesrank < products.salesrank;'''.format(asin))
    rows = cursor.fetchall()
    cursor.close()
    con.close()
    return rows
# getSimilarsWithMostSalesrank('0738700797')

### Questao 5c:

In [4]:
def getProductAvgTimeline(asin, con):
    cursor = con.cursor()
    cursor.execute('''select date,rating from products join reviews r on products.asin = r.product_asin where asin='{}' order by date;'''.format(asin))
    rows = cursor.fetchall()
    cursor.close()
    con.close()
    ratings = []
    for index in range(0,len(rows)):
        avg = 0
        for j in range(0,index+1):
            avg += rows[j][1]
        avg = avg/(index+1)
        ratings.append((rows[index][0],avg))
    return ratings
# getProductAvgTimeline('0738700797')

### Questao 5d:

In [5]:
def getTop10ProductsOfEachGroup(con):
    cursor = con.cursor()
    cursor.execute('''SELECT * FROM (
                SELECT *, rank() OVER (
                    PARTITION BY products."group"
                    ORDER BY products.salesrank DESC
                )
                FROM products
                WHERE salesrank notnull
                ) jobs WHERE  RANK <=10;''')
    rows = cursor.fetchall()
    cursor.close()
    con.close()
    return rows
# getTop10ProductsOfEachGroup()

### Questao 5e:

In [6]:
def getTop10ProductsWithMostRatingAVG(con):
    cursor = con.cursor()
    cursor.execute('''SELECT asin,title,avg(helpful) FROM products join reviews r on products.asin = r.product_asin 
    GROUP BY (asin,title) order by avg(helpful) DESC LIMIT 10;''')
    rows = cursor.fetchall()
    cursor.close()
    con.close()
    return rows
# getTop10ProductsWithMostRatingAVG()

### Questao 5f:

In [7]:
def getTop5CategoriesWithMostRating(con):
    cursor = con.cursor()
    cursor.execute('''select category_id,name,avg
        from (
         SELECT category_id, avg(helpful)
         FROM products
                  join reviews r on products.asin = r.product_asin
                  join product_category pc on products.asin = pc.product_asin
                  join categories c on pc.category_id = c.id
         group by (category_id)
         order by avg(helpful) desc
         limit 5
        ) result join categories on category_id=id;''')
    rows = cursor.fetchall()
    cursor.close()
    con.close()
    return rows
# getTop5CategoriesWithMostRating()

### Questao 5g:

In [8]:
def get10CustomersWithMostReviewsForEachGroup(con):
    cursor = con.cursor()
    cursor.execute('''SELECT customer,"group",count FROM (
                SELECT *, row_number()  OVER (
                    PARTITION BY "group"
                    ORDER BY count DESC
                )
                FROM ( SELECT count(customer),customer,"group" FROM reviews,products WHERE product_asin = asin GROUP BY customer,"group" ORDER BY "group",count(customer) DESC) xuei
            ) jobs WHERE row_number <= 10;''')
    rows = cursor.fetchall()
    cursor.close()
    con.close()
    return rows
# get10CustomersWithMostReviewsForEachGroup()

In [9]:
def create_connection(user, password, host, port, dbname):
    con = psycopg2.connect( user = user,
                        password = password,
                        host = host,
                        port = port,
                        dbname = dbname)
    return con

In [10]:
# print("Insira os parâmetros da conexão: Usuário, senha, endereço do servidor, porta, nome da database")
# print("Porta padrão é 5432")
# user = input("Usuário")
# password = input("Senha")
# host = input("Endereço")
# port = input("Porta")
# dbname = input("Nome da database")
user = "postgres"
password = "postgres"
host = "localhost"
port = "5432"
dbname = "amazon"

## Resultados:

In [14]:
#5 a)
con = create_connection(user,password,host,port,dbname)
result = getHelpfulReviews('0738700797', con)

In [12]:
#5 b)
con = create_connection(user,password,host,port,dbname)
result = getSimilarsWithMostSalesrank('0738700797',con)


In [ ]:
#5 c)
con = create_connection(user,password,host,port,dbname)
result = getProductAvgTimeline('0738700797',con)

In [ ]:
#5 d)
con = create_connection(user,password,host,port,dbname)
result = getTop10ProductsOfEachGroup(con)

In [ ]:
#5 e)
con = create_connection(user,password,host,port,dbname)
result = getTop10ProductsWithMostRatingAVG(con)

In [ ]:
#5 f)
con = create_connection(user,password,host,port,dbname)
result = getTop5CategoriesWithMostRating(con)

In [39]:
#5 g)
con = create_connection(user,password,host,port,dbname)
result = np.array(get10CustomersWithMostReviewsForEachGroup(con))

In [83]:
dictClt = {}
for i in result:
    if i[1] not in dictClt:
        dictClt[i[1]] = [[i[0],i[2]]]
    else:
        dictClt[i[1]].append([i[0],i[2]])

for group in dictClt:
    print("\n10 clientes com mais comentários para {} :".format(group))
    clts = dictClt[group]
    idx = 1
    for clt in clts:
        print("    {}o: {}".format(idx,clt[0]))
        idx += 1
        

# result = 


10 clientes com mais comentários para Book :
    1o: ATVPDKIKX0DER
    2o: A3UN6WX5RRO2AG
    3o: A14OJS0VWMOSWO
    4o: AFVQZQ8PW0L
    5o: A2BGGOF7MYBYOM
    6o: A1IX3X8IVQ237L
    7o: A3GBMAJNL0YQ1K
    8o: A2VNZA22F2QTD8
    9o: A1K1JW1C5CUSUZ
    10o: A2NJO6YE954DBH

10 clientes com mais comentários para DVD :
    1o: ATVPDKIKX0DER
    2o: A3UN6WX5RRO2AG
    3o: A2HS1WB42BT31I
    4o: A2DXBKKGZMCPS
    5o: A2NJO6YE954DBH
    6o: A3LZGLA88K0LA0
    7o: A2W00XKTTF1G0Z
    8o: A2XVRCU5DQBULH
    9o: AI0OAQ6E2O8VF
    10o: A375OT7C8ZJCNU

10 clientes com mais comentários para Music :
    1o: ATVPDKIKX0DER
    2o: A3UN6WX5RRO2AG
    3o: A2MOZZCTN2RJH3
    4o: A9Q28YTLYREO7
    5o: A2TLFXHJ4VMNL
    6o: A2U49LUUY4IKQQ
    7o: A2NJO6YE954DBH
    8o: A1J5KCZC8CMW9I
    9o: AZL0P9J95ON7D
    10o: AWFDFLR1DA23A

10 clientes com mais comentários para Toy :
    1o: A2U1T90IOVPBAR
    2o: AH16IHWEMA61J

10 clientes com mais comentários para Video :
    1o: ATVPDKIKX0DER
    2o: A3UN6WX5RRO2AG

[['ATVPDKIKX0DER', '5150'],
 ['A3UN6WX5RRO2AG', '887'],
 ['A14OJS0VWMOSWO', '71'],
 ['AFVQZQ8PW0L', '36'],
 ['A2BGGOF7MYBYOM', '24'],
 ['A1IX3X8IVQ237L', '20'],
 ['A3GBMAJNL0YQ1K', '20'],
 ['A2VNZA22F2QTD8', '18'],
 ['A1K1JW1C5CUSUZ', '15'],
 ['A2NJO6YE954DBH', '14']]